# Version Info
v5 version에서 cycleGAN 부분을 아래 notebook을 바탕으로 수정 중 == v6 version<br>https://colab.research.google.com/drive/1n5OzIaic1Ho60UAtqEgCnRyn7Lg1Po_v<br>sequence 길이를 모두 1로 맞춰야 함, 그렇지 않으면 생성할 때 마다 차원이 감소함. 아니면 train 과정에서 모델이 seq 길이 만큼 하나의 데이터에 대해 반복을 수행해서 생성을 진행해야 한다.

v6 version lstm cell return_sequnces=True로 설정하고 마지막 output layer에서 2차원 dense를 수행<br> --> seq_l을 데이터의 차원으로 갖도록 함

cycleGAN의 generation 부분 -> v7

cycleGAN의 loss 및 훈련 -> v8

gradients tape 문제 해결을 위한 tf dataset 이용 -> v9

v9의 cycleGAN pt type code 정리 및 파이프라인 보수 -> v10<br>
v10에서 seq화 과정에서 발생하는 데이터 손실, loss 그래프, gpu 사용량 최적화, notes to MIDI(생성 데이터 MIDI로 변환) 수정 -> v11<br>
v11에서 todo 6~ 까지의 수정 -> v12
v12에서 vocab size 관련 error 수정 + batch 등의 파라미터 조정 -> v13
v13에서 만든 pt type 완성본을 여러가지 방면으로 실험 -> v14

# TODO
1. 일반 LSTM model과는 다르게 cycleGAN의 input data를 만들어야 한다.
즉, sequence data로 data를 변환할 때, 일반 LSTM model과는 달리 target data를 생성하지 않아야 하며 seq_L 만큼 마다 나눠서 데이터 손실 없이 input data로 사용해야한다.

    -> 모델이 seq_l 길이 만큼의 데이터를 바탕으로 (그 길이 안 데이터 해석) output을 만들어 낸다.<br> 이후 sequence data로 나뉘어진 output data를 다시 reshape한 후, midi로 변환하면 된다.

2. sequence data 및 음악적 특성을 충분히 고려할 수 있도록 discriminator 및 그에 따른 loss function 구현 필요 -- 시간 계획 필수 (기한 고려)

3. 향후 cycleGAN과 관련된 loss(cycle consistnecy 등)도 손을 볼 수 있으면 좋을 것 같다.

4. GPU 사용량 늘리기

5. batch를 만들때, shuffle 주의하기, test data에는 사용하지 않아야 할수도 있다. -> train data shuffle을 수행해도 무방할 정도의 seq_l을 잡거나 shuffle을 수행하지 않아야 한다.

6. reshape 이용해서 create sequence 부분 고치기

7. 실제 연구 데이터를 사용할 때, 두 도메인의 데이터 길이 조심하기 - 학습과정에서

8. notes_to_midi 잘 이용하기

9. MIDI file 제대로, 전부 append 되는 지 확인하기 --> file_path glob 부분에서 slicing에 의한 문제 --> 해결

10. cycleGAN 생성 과정 중에 pitch, step, duration의 loss 차이가 튼 영향을 미친다면 music_generation 노트북을 활용하여 loss 재정의하기

11. 각 노래 midi 별로 나눠서 처리하거나, batch size 크기를 늘려 모델이 batch로 학습을 진행할 때, 각각의 노래가 섞이는 경우를 조금은 줄이는 것이 좋은지 실험 필요

12. colab 환경에서 여러개의 notebook을 띄워 놓은 후, 각각의 변수들을 변경하면서 실험 진행<br>

13. 12를 진행하는 과정에서 실험 리스트, 설계 과정, 결과 등을 모두 기록해야함

14. colab notebook 최대 가용량 -> 실험 가능량 설계 -> 시간 계획



# Experiment plan
1. batch
2. data<br>
    1. count
    2. type<br>
        1. musician<br>
            1. piano-e-competition
        2. elec<br>
            1. maestro
3. generator model<br>
    1. model structure<br>
        1. type<br>
            1. bidirectional
            2. Dense
            3. GRU
        2. layer<br>
            1. unit count
    2. hyper parameter
4. discriminator model
    1. model structure<br>
        1. type<br>
            1. bidirectional
            2. Dense
            3. GRU
        2. layer<br>
            1. unit count
    2. hyper parameter<br>
    3. for music<br>
        1. patchGAN
5. loss function weight<br>
    1. brute force method
6. optimizer<br>
    1. type
    2. learning rate
7. future plan<br>
    1. 특정곡의 곡 또는 작곡관련 nlp 정보 -> embbding -> 새로운 층으로 concatenate -> multimodal<br>
    2. 

In [1]:
# for installing

!pip install -U -q pip
!pip install -q tensorboardX
!pip install -q git+https://github.com/tensorflow/examples.git
!pip install -q pretty_midi
!pip install -q knockknock

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/build_env.py", line 20, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.7/dist-

In [1]:
!git clone https://github.com/chelate1118/piano-e-competition-midi-files.git

Cloning into 'piano-e-competition-midi-files'...
Updating files:   4% (241/5486)
Updating files:   5% (275/5486)
Updating files:   6% (330/5486)
Updating files:   7% (385/5486)
Updating files:   8% (439/5486)
Updating files:   9% (494/5486)
Updating files:  10% (549/5486)
Updating files:  10% (570/5486)
Updating files:  11% (604/5486)
Updating files:  12% (659/5486)
Updating files:  13% (714/5486)
Updating files:  14% (769/5486)
Updating files:  15% (823/5486)
Updating files:  15% (864/5486)
Updating files:  16% (878/5486)
Updating files:  17% (933/5486)
Updating files:  18% (988/5486)
Updating files:  19% (1043/5486)
Updating files:  20% (1098/5486)
Updating files:  21% (1153/5486)
Updating files:  21% (1197/5486)
Updating files:  22% (1207/5486)
Updating files:  23% (1262/5486)
Updating files:  24% (1317/5486)
Updating files:  25% (1372/5486)
Updating files:  25% (1388/5486)
Updating files:  26% (1427/5486)
Updating files:  27% (1482/5486)
Updating files:  28% (1537/5486)
Updating fi

In [2]:
import sys
print(sys.executable)

c:\Users\gobak\.virtualenvs\.venv-HOEqjjdC\Scripts\python.exe


In [5]:
# import os 
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [3]:
# import 과정 error -> AVX2를 이용한 tf compile 진행 시, 더 최적화 가능함을 알려줌 | 아래 링크 참조
# http://erpcomputing.com/build-tensorflow-from-source--mkl-enabled

#library import

# for data and preprocessing
import tensorflow_datasets as tfds
import collections
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
from sklearn.model_selection import train_test_split
# import MIDI_Pedal # 사용법 : MIDI_Pedal.apply('test.mid', 'output.mid')


# for model
# 설치가 git인 경우 local에서 설치한 후 .venv/Lib에 넣으면 됨, local pip 설치경로는 위에 있음
import tensorflow as tf

# for the others
# import fluidsynth # MIDI는 다른 거로 재생하기
import datetime
from IPython import display
from IPython.display import clear_output
import matplotlib.pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple
import time
import os
# from knockknock import desktop_sender
# from google.colab import files # google colab에서 결과 midi file 다운받고 싶을 때 import 하면 됨.

In [4]:
# 단순 model test용 데이터

data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

sample_file = filenames[0]
print(sample_file)

Number of files: 1282
data\maestro-v2.0.0\2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi


In [5]:
pm = pretty_midi.PrettyMIDI(sample_file)

print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

Number of instruments: 1
Instrument name: Acoustic Grand Piano


In [6]:
# global variable declaration

#global
seed = 42

# for model
AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(seed)
BUFFER_SIZE = -1
BATCH_SIZE = 1 # cycleGAN tf document 참고한 값 
SEQ_LENGTH = 512
VOCAB_SIZE = 128

N_FEATURE = OUTPUT_CHANNELS = 3
LR = 2e-4 # opt
B1 = 0.5 # opt

# for data
np.random.seed(seed)
# MUSICIAN_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')
MUSICIAN_MIDI_FILES_DIR = pathlib.Path('piano-e-competition-midi-files/midi')
ELEC_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')

# for the others
# Sampling rate for audio playback
_SAMPLING_RATE = 16000
PRINT_ITER = 50

In [ ]:

N_FEATURE = OUTPUT_CHANNELS = 3
LR = 2e-4 # opt
B1 = 0.5 # opt

# for data
np.random.seed(seed)
# MUSICIAN_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')
MUSICIAN_MIDI_FILES_DIR = pathlib.Path('piano-e-competition-midi-files/midi')
ELEC_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')

# for the others
# Sampling rate for audio playback
_SAMPLING_RATE = 16000
PRINT_ITER = 50

In [7]:
musician_filenames = glob.glob(str(MUSICIAN_MIDI_FILES_DIR/'*.mid*'))

용어 정리<br>real : 실제 연주자 연주<br>elec : 전자 음악 연주

In [8]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

def plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None):
    if count:
      title = f'First {count} notes'
    else:
      title = f'Whole track'
      count = len(notes['pitch'])
    plt.figure(figsize=(20, 4))
    plot_pitch = np.stack([notes['pitch'], notes['pitch']], axis=0)
    plot_start_stop = np.stack([notes['start'], notes['end']], axis=0)
    plt.plot(
        plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
    plt.xlabel('Time [s]')
    plt.ylabel('Pitch')
    _ = plt.title(title)

def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5):
  plt.figure(figsize=[15, 5])
  plt.subplot(1, 3, 1)
  sns.histplot(notes, x="pitch", bins=20)

  plt.subplot(1, 3, 2)
  max_step = np.percentile(notes['step'], 100 - drop_percentile)
  sns.histplot(notes, x="step", bins=np.linspace(0, max_step, 21))
  
  plt.subplot(1, 3, 3)
  max_duration = np.percentile(notes['duration'], 100 - drop_percentile)
  sns.histplot(notes, x="duration", bins=np.linspace(0, max_duration, 21))

def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

##### 참고
<span style='font-size:80%'>1. midi_to_notes(midi_file: str) -> pd.DataFrame<br>2. notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI<br>3. plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None)<br>4. def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5)</span>

In [9]:
musician_filenames = glob.glob(str(ELEC_MIDI_FILES_DIR/'**/*.mid*'))

len(musician_filenames)

1282

In [10]:
# data import musician AND FAKE
# error 많이 나면 np.array 이용하기
# filenames slicing 수 적절히 설정하기

# musician_filenames = glob.glob(str(MUSICIAN_MIDI_FILES_DIR/'**/*.mid*'))
musician_filenames = glob.glob(str(MUSICIAN_MIDI_FILES_DIR/'*.mid*'))
elec_filenames = glob.glob(str(ELEC_MIDI_FILES_DIR/'**/*.mid*'))


musician_all_notes = []
elec_all_notes = []

# 각 notes listz
# 0 : musician - 사람이 연주한 데이터
# 1 : fake - 사람이 연주하지 않은 데이터
data_notes = [musician_all_notes, elec_all_notes]
data_paths = [musician_filenames, elec_filenames]
data_notes_ds = {} # tfds를 이용한 slices 이후 데이터임, (n_notes, tf_dt) tuple형의 데이터가 원소인 list

train_data_notes_ds = [] # n_notes, dataset(slice)의 tuple 형태
validation_data_notes_ds = []
test_data_notes_ds = []

data_split_ratio = {'train' : 0.8, 'validation' : 0.2, 'test' : 0.2}

key_order = ['pitch', 'step', 'duration']
not_key_order = ['start', 'end']

data_type = ['musician', 'elec']

# notes 수를 맞추기 위함
n_notes = 10e9

file_cnt = 0

print(f'{len(musician_filenames) + len(elec_filenames)} files are ready to process')
print('-' * PRINT_ITER)
print()
print('Start!')
print()
time.sleep(2)

start_time = time.time()

for i, data_path in enumerate(data_paths):
    for f in data_path:
        notes = midi_to_notes(f)
        data_notes[i].append(notes)
        if file_cnt % 10 == 0:
            clear_output(wait=True)
            print(f'process {(file_cnt / (len(musician_filenames) + len(elec_filenames))) * 100.0:.3f}% {file_cnt}th file process continue...')
        file_cnt += 1
        
    # append와 concat은 다르다!
    data_notes[i] = pd.concat(data_notes[i])
    data_notes[i].drop(columns=not_key_order, inplace=True)
    n_notes = min(len(data_notes[i]), n_notes)    
    
for i, data_path in enumerate(data_paths):
    _train, _test = data_notes[i][:int(n_notes * (1 - data_split_ratio['test']))], data_notes[i][int(n_notes * (1 - data_split_ratio['test'])):]
    _train, _validation = _train[:int(len(_train) * data_split_ratio['train'])], _train[int(len(_train) * data_split_ratio['train']):]
    n__train = len(_train)
    n__validation = len(_validation)
    n__test = len(_test)
    data_dict = {'train' : (n__train, _train), 'validation' : (n__validation, _validation), 'test' : (n__test, _test)}
    data_notes_ds[data_type[i]] = data_dict
    
    
print()
print('End!')

print(f'total process time for {len(musician_filenames) + len(elec_filenames)} files : {time.time() - start_time:.3f}sec')
# data_notes_ds Info
# (n_data, tfds) 원소,
# musician ==> train, validation, test
# elec ==> train, validation, test
# 총 6개의 원소가 들어가 있음

process 99.950% 4010th file process continue...

End!
total process time for 4012 files : 2576.241sec


In [11]:
n_notes

7126318

In [12]:
data_notes_ds['musician']['train'][1]

,pitch,step,duration
0,71,0.000000,2.545933
1,67,0.002137,2.543796
2,59,0.002137,2.541659
3,50,0.004273,2.537386
4,43,0.006410,2.530975
...,...,...,...
3818,76,0.002671,1.071046
3819,73,0.004006,0.066773
3820,54,0.002671,0.064103
3821,62,0.009348,1.055020


In [13]:
data_notes_ds['elec']['train'][1]

,pitch,step,duration
0,71,0.000000,0.096875
1,55,0.186458,0.217708
2,71,0.009375,0.505208
3,59,0.175000,0.167708
4,62,0.169792,0.119792
...,...,...,...
8359,53,0.082031,0.157552
8360,51,0.160156,0.036458
8361,79,0.014323,0.115885
8362,80,0.134115,0.131510


In [14]:
len(data_notes_ds['elec']['train'][1]) + len(data_notes_ds['elec']['test'][1]) + len(data_notes_ds['elec']['validation'][1])

7126318

In [15]:
len(data_notes_ds['musician']['train'][1]) + len(data_notes_ds['musician']['test'][1]) + len(data_notes_ds['musician']['validation'][1])

14341631

In [ ]:
# SEQ_LENGTH = 128

# cycleGAN

In [16]:
# v2
def get_generator(model_name, output_channels=OUTPUT_CHANNELS):
  input_shape = (SEQ_LENGTH, output_channels)
  # print(input_shape)
  inputs = tf.keras.Input(input_shape)

  x0 = tf.keras.layers.LSTM(524, return_sequences=True)(inputs)
  x1 = tf.keras.layers.LSTM(256, return_sequences=True)(x0)
  x2 = tf.keras.layers.LSTM(128, return_sequences=True)(x1)
  x3 = tf.keras.layers.LSTM(SEQ_LENGTH, return_sequences=True)(x2)
  outputs = tf.keras.layers.Dense(3, name='all')(x3)

  model = tf.keras.Model(inputs, outputs, name=model_name)
  return model


# 향후 수정 후, 사용하기
def get_discriminator(model_name, output_channels=OUTPUT_CHANNELS):
    input_shape = (SEQ_LENGTH, output_channels)
    inputs = tf.keras.Input(input_shape)
    x0 = tf.keras.layers.LSTM(524, return_sequences=True)(inputs)
    x1 = tf.keras.layers.LSTM(256, return_sequences=True)(x0)
    x2 = tf.keras.layers.LSTM(128, return_sequences=True)(x1)
    outputs = tf.keras.layers.LSTM(SEQ_LENGTH, return_sequences=True)(x2)

    model = tf.keras.Model(inputs, outputs, name=model_name)
    return model
    

# 향후 종류를 다양하게 변형하기
gen_musician = get_generator('gen_musician')
gen_elec = get_generator('gen_elec')

disc_musician = get_discriminator('disc_musician')
disc_elec = get_discriminator('disc_elec')

In [17]:
gen_musician.summary()

Model: "gen_musician"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 3)]          0         
                                                                 
 lstm (LSTM)                 (None, 512, 524)          1106688   
                                                                 
 lstm_1 (LSTM)               (None, 512, 256)          799744    
                                                                 
 lstm_2 (LSTM)               (None, 512, 128)          197120    
                                                                 
 lstm_3 (LSTM)               (None, 512, 512)          1312768   
                                                                 
 all (Dense)                 (None, 512, 3)            1539      
                                                                 
Total params: 3,417,859
Trainable params: 3,417,859
No

In [18]:
# loss function

loss_obj = tf.keras.losses.BinaryCrossentropy()
# losses_ratio = {'disc_loss' : 1.0, 'gen_loss' : 1.0, 'cycle_loss' : 1.0}

def disc_loss(real, generated):

    real_loss = loss_obj(tf.ones_like(real), real)
    fake_loss = loss_obj(tf.zeros_like(generated), generated)
    
    total_loss = real_loss + fake_loss

    return total_loss

def gen_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

def cycle_loss(real_data, cycled_data):
    return tf.reduce_mean(tf.abs(real_data - cycled_data))

def iden_loss(real_data, gen_from_real_data):
    return tf.reduce_mean(tf.abs(real_data - gen_from_real_data))

In [19]:
# optimizer

# LR = 2e-4
# B1 = 0.5 

gen_musician_opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=B1)
gen_elec_opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=B1)

disc_musician_opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=B1)
disc_elec_opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=B1)

In [30]:
# checkpoint_path의 경우 계속해서 바꿔줘야 에러가 나지 않음, 이 cell(+위에 model 정의 cell)을 반복해서 실행하면 error 발생함
checkpoint_path = "./checkpoint/train_2022_11_10"

ckpt = tf.train.Checkpoint(gen_musician=gen_musician,
                           gen_elec=gen_elec,
                           disc_musician=disc_musician,
                           disc_elec=disc_elec,
                           gen_musician_opt=gen_musician_opt,
                           gen_elec_opt=gen_elec_opt,
                           disc_musician_opt=disc_musician_opt,
                           disc_elec_opt=disc_elec_opt)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [20]:
# real_x : musician data
# real_y : elec data
# g : gen elec
# f : gen music

@tf.function
def train_step(real_x, real_y, losses_ratio):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    all_loss = []
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.
        fake_y = gen_elec(real_x, training=True)
        cycled_x = gen_musician(fake_y, training=True)

        fake_x = gen_musician(real_y, training=True)
        cycled_y = gen_elec(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = gen_musician(real_x, training=True)
        same_y = gen_elec(real_y, training=True)


        disc_real_x = disc_musician(real_x, training=True)
        disc_real_y = disc_elec(real_y, training=True)

        disc_fake_x = disc_musician(fake_x, training=True)
        disc_fake_y = disc_elec(fake_y, training=True)

        # calculate the loss
        gen_elec_loss = gen_loss(disc_fake_y)
        gen_musician_loss = gen_loss(disc_fake_x)
        total_cycle_loss = cycle_loss(real_x, cycled_x) + cycle_loss(real_y, cycled_y)

        elec_iden_loss = iden_loss(real_y, same_y)
        musician_iden_loss = iden_loss(real_x, same_x)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_elec_loss = gen_elec_loss*losses_ratio['gen'] + total_cycle_loss*losses_ratio['cyc'] + elec_iden_loss*losses_ratio['iden']
        total_gen_musician_loss = gen_musician_loss*losses_ratio['gen'] + total_cycle_loss*losses_ratio['cyc'] + musician_iden_loss*losses_ratio['iden']

        disc_musician_loss = disc_loss(disc_real_x, disc_fake_x)*losses_ratio['disc']
        disc_elec_loss = disc_loss(disc_real_y, disc_fake_y)*losses_ratio['disc']

        all_loss = [tf.identity(gen_elec_loss), 
                    tf.identity(gen_musician_loss), 
                    tf.identity(total_cycle_loss), 
                    tf.identity(total_gen_elec_loss),
                    tf.identity(total_gen_musician_loss),
                    tf.identity(disc_musician_loss),
                    tf.identity(disc_elec_loss),
                    tf.identity(musician_iden_loss),
                    tf.identity(elec_iden_loss)]


    # Calculate the gradients for generator and discriminator
    gen_elec_gradients = tape.gradient(total_gen_elec_loss, 
                                          gen_elec.trainable_variables)
    
    gen_musician_gradients = tape.gradient(total_gen_musician_loss, 
                                          gen_musician.trainable_variables)
    
    disc_musician_gradients = tape.gradient(disc_musician_loss, 
                                              disc_musician.trainable_variables)
    disc_elec_gradients = tape.gradient(disc_elec_loss, 
                                              disc_elec.trainable_variables)

    # Apply the gradients to the optimizer
    gen_elec_opt.apply_gradients(zip(gen_elec_gradients, 
                                              gen_elec.trainable_variables))

    gen_musician_opt.apply_gradients(zip(gen_musician_gradients, 
                                              gen_musician.trainable_variables))
    
    disc_musician_opt.apply_gradients(zip(disc_musician_gradients,
                                                  disc_musician.trainable_variables))
    
    disc_elec_opt.apply_gradients(zip(disc_elec_gradients,
                                                  disc_elec.trainable_variables))
    
    return all_loss
    

In [21]:
def notes_to_trainable_dataset(all_notes, n_data):
    key_order = ['pitch', 'step', 'duration']
    train_notes = np.stack([all_notes[key] for key in key_order], axis=1)
    n_notes, n_features = train_notes.shape 

    train_notes = train_notes[:n_notes - (n_notes % SEQ_LENGTH)]
    train_notes[:, 0] = train_notes[:, 0] / VOCAB_SIZE

    seq_train_notes = train_notes.reshape(-1, SEQ_LENGTH, 3)
    # seq_train_notes = seq_train_notes / np.array([VOCAB_SIZE, 0.0, 0.0])

    print(f'seq_train_notes shape {seq_train_notes.shape}')
    print()

    seq_train_notes = seq_train_notes[:n_data]
    
    seq_ds = tf.data.Dataset.from_tensor_slices(seq_train_notes)
    print(f'notes_ds element spec {seq_ds.element_spec}')
    print()

    # BUFFER_SIZE = n_notes - SEQ_LENGTH
    train_ds = seq_ds.map(lambda x: (tf.cast(x, tf.float32)), num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    print(f'train_ds spec {train_ds.element_spec}')
    print()

    print('-' * PRINT_ITER)

    return train_ds


musician_train_ds = notes_to_trainable_dataset(data_notes_ds['musician']['train'][1], 100)

elec_train_ds = notes_to_trainable_dataset(data_notes_ds['elec']['train'][1], 100)

seq_train_notes shape (8907, 512, 3)

notes_ds element spec TensorSpec(shape=(512, 3), dtype=tf.float64, name=None)

train_ds spec TensorSpec(shape=(None, 512, 3), dtype=tf.float32, name=None)

--------------------------------------------------
seq_train_notes shape (8907, 512, 3)

notes_ds element spec TensorSpec(shape=(512, 3), dtype=tf.float64, name=None)

train_ds spec TensorSpec(shape=(None, 512, 3), dtype=tf.float32, name=None)

--------------------------------------------------


In [22]:
# @tf.function을 이용하여 code를 실행하는 도중에 .nmupy()와 같은 code 실행 가능
# https://github.com/tgjeon/TF-Eager-Execution-Guide-KR/blob/master/guide.md 참고
# https://www.tensorflow.org/api_docs/python/tf/config/run_functions_eagerly
tf.config.run_functions_eagerly(True)

In [24]:
from tensorboardX import SummaryWriter
writer = SummaryWriter('runs_test/')

In [23]:
EPOCHS = 1

In [37]:
2 * (35631 / 128)

556.734375

In [38]:
2 * (35631 / BATCH_SIZE)

5938.5

In [25]:
# tensorboard graph color rgb : #1fddff

losses_ratio = {'disc' : 1.0, 'gen' : 1.0, 'cyc' : 1.0, 'iden' : 1.0} # cycle, iden loss ratio는 서로 종속적임(in tf document)
all_loss = []
all_loss_name = ['gen_elec_loss', 
                 'gen_musician_loss',
                 'total_cycle_loss', 
                 'total_gen_elec_loss',
                 'total_gen_musician_loss',
                 'disc_musician_loss',
                 'disc_elec_loss',
                 'musician_iden_loss',
                 'elec_iden_loss']

%load_ext tensorboard
%tensorboard --logdir runs_test --port=6006

# 단순히 모델의 가중치 업데이트 횟수를 의미함
# 향후 이 값을 바탕으로 epoch를 계산 가능 | 
# update_n = epoch * (n_notes / (seq_length * batch_size)) = epoch * (n_seq_notes / batch_size)

update_cnt = 0

total_start_time = time.time()

for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    
    for musician, elec in tf.data.Dataset.zip((musician_train_ds, elec_train_ds)):
        # musician = tf.cast(musician, dtype=tf.float32)
        # elec = tf.cast(elec, dtype=tf.float32)
                
        all_loss = [0 for _ in range(len(all_loss_name))]   

        for_all_loss = train_step(musician, elec, losses_ratio)
        for i, loss in enumerate(for_all_loss):
            all_loss[i] += loss.numpy().item()

        if update_cnt % 5 == 0:
            print('-', end='')
        
        n += 1
        update_cnt += 1

        # 아래 부분에서 BATCH_SIZE로 나눌 지에 관해 고민 필요
        all_loss = [loss for loss in all_loss]
        for loss, loss_name in zip(all_loss, all_loss_name):
            writer.add_scalar(loss_name, loss, update_cnt)

    # clear_output(wait=True)
    
    # note를 시각화할 수 있는 거 넣기
    # ex) generate_images(generator_g, sample_horse)
    
    if (epoch + 1) % 3 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'Saving checkpoint for each {epoch + 1} at {ckpt_save_path}')
    
    print(f'Time taken for epoch {epoch + 1} is {time.time() - start:.3f} sec\n')

print()
print('-' * PRINT_ITER)
print(f'Time taken for total {EPOCHS} epoch : {time.time() - total_start_time:.3f} sec')
writer.close()

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2022-11-21 08:18:52.827729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-11-21 08:18:52.834120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-21 08:18:58.660702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-11-21 08:18:58.663898: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-21 08:18:58.683424: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-JSMT29H
2022-11-21 08:18:58.685637: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-JSMT29H
Traceback (most recent call last)

-

KeyboardInterrupt: 

In [ ]:
def generate_domain_notes(input_ds, model):
    result_list = []
    generated_notes_list = []
    n_batch_data = tf.data.experimental.cardinality(input_ds).numpy().item()
    # n_batch_data = 1

    for inp in input_ds.take(n_batch_data):
        gen_music = model(inp)
        result_list.append(gen_music)

    for result in result_list:
        pitchs = result[:, :, 0]
        steps = result[:, :, 1]
        durations = result[:, :, 2]

        pitchs = tf.reshape(pitchs, [-1])
        pitchs = tf.math.scalar_mul(VOCAB_SIZE, pitchs)
        pitchs = tf.math.round(pitchs)
        pitchs = tf.cast(pitchs, dtype=tf.int32)
        pitchs = pitchs.numpy()

        durations = tf.reshape(durations, [-1])
        durations = durations.numpy()

        steps = tf.reshape(steps, [-1])
        steps = steps.numpy() 

        generated_notes = []
        prev_start = 0

        for pitch, duration, step in zip(pitchs, durations, steps):
            pitch, duration, step = pitch.item(), duration.item(), step.item()

            if pitch > VOCAB_SIZE - 1:
                pitch = VOCAB_SIZE - 1
            if pitch < 0:
                pitch = 0
                
            start = prev_start + step 
            end = start + duration
            input_note = (pitch, step, duration)
            generated_notes.append((*input_note, start, end))

            prev_start = start

        generated_notes = pd.DataFrame(
            generated_notes, columns=(*key_order, 'start', 'end')
        )

        generated_notes_list.append(generated_notes)

    generated_notes_all = pd.concat(generated_notes_list, ignore_index=True)
    return generated_notes_all

def notes_to_testable_dataset(all_notes):
    key_order = ['pitch', 'step', 'duration']
    test_notes = np.stack([all_notes[key] for key in key_order], axis=1)
    n_notes, n_features = test_notes.shape 
    
    n_notes_for_seq = n_notes - (n_notes % SEQ_LENGTH)
    test_notes = test_notes[:n_notes_for_seq]
    
    test_notes = test_notes.reshape(-1, SEQ_LENGTH, 3)
    print(f'test_notes shape {test_notes.shape}')
    print()
    
    notes_ds = tf.data.Dataset.from_tensor_slices(test_notes)
    print(f'notes_ds element spec {notes_ds.element_spec}')
    print()

    seq_ds = notes_ds
    print(f'seq_ds element_spec {seq_ds.element_spec}')
    print()

    # BUFFER_SIZE = n_notes - SEQ_LENGTH
    test_ds = seq_ds.batch(n_notes_for_seq, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    print(f'test_ds spec {test_ds.element_spec}')
    print()

    print('-' * PRINT_ITER)

    return test_ds

def generate_notes(midi_filepath, model):
    notes = midi_to_notes(midi_filepath)
    test_ds = notes_to_testable_dataset(notes)
    generated_notes = generate_domain_notes(test_ds, model)
    return generated_notes
    

midi_filepath = elec_filenames[0]

generated_notes = generate_notes(midi_filepath, gen_musician)
generated_notes

In [26]:
!nvidia-smi

Sun Nov 20 10:37:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |   2810MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
out_file = 'output.mid'
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)

In [ ]:
# # generator 불러오기
# gen_musician_ckpt_path = ''
# gen_musician.load_weights(gen_musician_ckpt_path)

# gen_elec_ckpt_path = '' 
# gen_elec.load_weights(gen_elec_ckpt_path)